# Kepler problem

In [ ]:
using OrdinaryDiffEq
using LinearAlgebra
using ForwardDiff
using Plots

### Equations of motion

In cartesian coordinates

$$
\mathcal{H} = \dfrac{1}{2}\big( p_1^2 + p_2^2 \big) + \dfrac{1}{\sqrt{x_1^2 + x_2^2}}
$$

In [ ]:
H(p, q) = 0.5*norm(p)^2 - inv(norm(q))

pdot(dp, p, q, params, t) = ForwardDiff.gradient!(dp, q -> -H(p, q), q)
qdot(dq, p, q, params, t) = ForwardDiff.gradient!(dq, p -> H(p, q), p)

### Bounded orbits. Initial conditions

In cartesian coordinates

$$
\vec{x}_0 = (1 - \epsilon, 0) \qquad \vec{p}_0 = \bigg( 0, \sqrt{\dfrac{1 + \epsilon}{1 - \epsilon}} \bigg)
$$

In [ ]:
eps = 0.28

q0 = [1 - eps, 0.0]
p0 = [0.0, sqrt((1 + eps)/(1 - eps))]

### Time scale

In [ ]:
N = 120     # points per orbital period
K = 5400    # total number of cycles
M = 5       # number of last cycles to analyze

t_step = 2*pi/N
t_max = 2*pi*K

### Numerical solution

In [ ]:
prob = DynamicalODEProblem(pdot, qdot, p0, q0, (0.0, t_max))
sol = solve(prob, KahanLi6(), dt = t_step);
data = reduce(hcat, sol.u[N*(K-M):end])'

### Orbit

In [ ]:
Plots.scalefontsizes()
Plots.scalefontsizes(0.5)
plotOrbit = scatter(
    data[:, 3], data[:, 4],
    aspect_ratio = 1,
    xlabel = "x",
    ylabel = "y",
    plot_title = "Kepler problem",
    markersize = 0.5,
    alpha = 0.25,
    legend = false,
    dpi = 300
)
annotate!(
    xlims(plotOrbit)[1] + 0.12*(xlims(plotOrbit)[2] - xlims(plotOrbit)[1]),
    ylims(plotOrbit)[2] - 0.03*(ylims(plotOrbit)[2] - ylims(plotOrbit)[1]),
    text("\\epsilon = $(eps)", :gray, :right, 4)
)

### Integration error

In [ ]:
Plots.scalefontsizes()
Plots.scalefontsizes(0.5)
plotEnergy = scatter(
    sol.t[N*(K-M):end]/(2*pi),
    map(eachrow(data)) do s
        H([s[1], s[2]], [s[3], s[4]]) - H(p0, q0)
    end,
    xlabel = "Rotations - N",
    ylabel = "Energy error - \\Delta E",
    plot_title = "Kepler problem",
    markersize = 0.5,
    alpha = 0.25,
    legend = false,
    dpi = 300
)